In [250]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import time
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning

# Ignore warnings below
pd.options.mode.chained_assignment = (
    None  # default='warn' # Remove copy on slice warning
)

Graph formation without HNS

In [231]:
def graph_formation(
    n_patients,
    n_doctors,
    max_number_connections,
    z=1.4,
    type_distance="default",
    beta_age_p_graph=0.01,
    beta_age_d_graph=0.01,
    beta_sex_p_graph=0.5,
    beta_sex_d_graph=0.5,
    beta_distance_graph=-0.5,
    alpha_law_graph=(-1, 1),
    psi_law_graph=(-1, 1),
    nb_latent_factors=None,
):
    """
    Creates only the graph formation part and returns the associated dataframe
    """
    coor_patients = []
    coor_doctors = []
    alpha_graph = []
    psi_graph = []
    rng = np.random.default_rng(None)
    D = np.zeros([n_patients, n_doctors], dtype=np.ndarray)

    if nb_latent_factors == None:  # We pick a random number of latent factors
        random_number = random.randint(1, 10)
        for i in range(n_patients):

            # We generate the FE for the graph formation model
            alpha_graph.append(
                np.random.uniform(
                    alpha_law_graph[0], alpha_law_graph[1], size=random_number
                )
            )

        for j in range(n_doctors):

            # We generate the FE for the graph formation model
            psi_graph.append(
                np.random.uniform(
                    psi_law_graph[0], psi_law_graph[1], size=random_number
                )
            )

    else:

        for i in range(n_patients):

            # We generate the FE for the graph formation model
            alpha_graph.append(
                np.random.uniform(
                    alpha_law_graph[0], alpha_law_graph[1], size=nb_latent_factors
                )
            )

        for j in range(n_doctors):

            # We generate the FE for the graph formation model
            psi_graph.append(
                np.random.uniform(
                    psi_law_graph[0], psi_law_graph[1], size=nb_latent_factors
                )
            )

    # Generate distance matrix
    if type_distance == "default":
        for i in range(n_patients):
            # Generate the coordinates of the patients
            coor_patients.append(np.random.uniform(0, 1, 2))
            for j in range(n_doctors):
                if (
                    i == 0
                ):  # We ensure each coordinate is generated once for each doctor
                    # Generate the coordinates of the doctors
                    coor_doctors.append(np.random.uniform(0, 1, 2))
                d = np.sqrt(
                    np.power((coor_patients[i][0] - coor_doctors[j][0]), 2)
                    + np.power((coor_patients[i][1] - coor_doctors[j][1]), 2)
                )
                D[i][j] = d

    else:

        # Assign randomly a CODGEO, DEP, or REG to patients and doctors
        dist_matrix = pd.read_csv("../Data/" + type_distance + ".csv")

        del dist_matrix[dist_matrix.columns[0]]
        dist_matrix.index = dist_matrix.columns
        for i in range(len(dist_matrix)):
            dist_matrix.iloc[i, i] = 0
        arr = dist_matrix.columns.values
        for i, col in enumerate(arr):
            arr[i] = int(float(arr[i]))
        dist_matrix.columns = arr
        dist_matrix.index = arr

        # Generate code for patient and doctor
        code_patient = []
        code_doctor = []
        for i in range(n_patients):
            random_code = np.random.choice(dist_matrix.columns.values)
            code_patient.append(random_code)
        for j in range(n_doctors):
            random_code = np.random.choice(dist_matrix.columns.values)
            code_doctor.append(random_code)
        for i in range(n_patients):
            for j in range(n_doctors):
                D[i, j] = dist_matrix.loc[code_patient[i], code_doctor[j]]

    D_normed = (D - D.mean()) / D.std()
    # Random draws of ages for patients and doctors
    sim_patient_age = rng.integers(low=1, high=99, size=n_patients)
    sim_patient_age_normed = (
        sim_patient_age - sim_patient_age.mean()
    ) / sim_patient_age.std()
    sim_doctor_age = rng.integers(low=26, high=99, size=n_doctors)
    sim_doctor_age_normed = (
        sim_doctor_age - sim_doctor_age.mean()
    ) / sim_doctor_age.std()

    # Random draws of genders of patients and doctors
    sim_patient_gender = np.random.choice(np.array([1, 2]), n_patients)
    sim_doctor_gender = np.random.choice(np.array([1, 2]), n_doctors)
    sim_patient_gender_normed = (
        sim_patient_gender - sim_patient_gender.mean()
    ) / sim_patient_gender.std()
    sim_doctor_gender_normed = (
        sim_doctor_gender - sim_doctor_gender.mean()
    ) / sim_doctor_gender.std()

    # Compile ids
    id_p = np.repeat(range(n_patients), n_doctors)
    id_d = np.tile(range(n_doctors), n_patients)

    # Compile observed features
    age_p_data = np.repeat(sim_patient_age, n_doctors)
    age_d_data = np.tile(sim_doctor_age, n_patients)
    sex_p_data = np.repeat(sim_patient_gender, n_doctors)
    sex_d_data = np.tile(sim_doctor_gender, n_patients)
    # bias_patient_data = np.repeat(bias_patient, n_doctors)
    # bias_doctor_data = np.tile(bias_doctor, n_patients)
    ef_patient_data = np.repeat(alpha_graph, n_doctors)
    ef_doctor_data = np.tile(psi_graph, n_patients)
    if type_distance != "default":
        code_patient_data = np.repeat(code_patient, n_doctors)
        code_doctor_data = np.tile(code_doctor, n_patients)
    # # P is the matrix with all the connection probabilities
    # P = np.zeros((n_patients, n_doctors))
    # Generate the identifier matrix A based on the distance
    A = np.zeros([n_patients, n_doctors], dtype=np.ndarray)
    for i in range(n_patients):
        for j in range(n_doctors):
            if (
                D[i][j] <= z
            ):  # if patient i and doctor j are too far away, there is no relation
                T = (
                    np.dot(alpha_graph[i], psi_graph[j])
                    + beta_age_p_graph * sim_patient_age_normed[i]
                    + beta_age_d_graph * sim_doctor_age_normed[j]
                    + beta_sex_p_graph * sim_patient_gender_normed[i]
                    + beta_sex_d_graph * sim_doctor_gender_normed[j]
                    + beta_distance_graph * D[i][j]
                )
                p = 1 / (1 + np.exp(-T))
                # P[i][j] = p
                A[i][j] = np.random.binomial(1, p)

    # Compile relations between doctors and patients
    relation = A.flatten()

    # Merge all columns into a dataframe
    dataframe = pd.DataFrame(
        data={
            "i": id_p,
            "j": id_d,
            "y": relation,
            "age_p": age_p_data,
            "age_d": age_d_data,
            "sex_p": sex_p_data,
            "sex_d": sex_d_data,
        }
    )
    dataframe["distance"] = D[dataframe["i"], dataframe["j"]].astype(float)

    # Now, we bound the number of connections (1 <= connections <= max_number_connections)
    # First, we detect the patients who have 0 connection.
    number_of_connections = dataframe.groupby("i").agg({"y": "sum"})
    zero_connection = number_of_connections[number_of_connections["y"] == 0].index
    for patient in zero_connection:
        # If patient has zero connection, we connect him with the nearest doctor (even if the threshold z isn't respected)
        min_index = dataframe[dataframe["i"] == patient]["distance"].idxmin()
        doctor_to_connect = dataframe.loc[min_index, "j"]
        dataframe.loc[
            (dataframe["i"] == patient) & (dataframe["j"] == doctor_to_connect), "y"
        ] = 1

    # Then, we detect the patients who have more than max_number_connections. We choose the remaining connections by doctor's popularities (possible to choose randomly).
    number_of_connections = dataframe.groupby("i").agg({"y": "sum"})
    too_much_connection = number_of_connections[
        number_of_connections["y"] > max_number_connections
    ].index
    for patient in too_much_connection:
        # We keep the connections with the most popular doctors
        patient_df = dataframe[dataframe["i"] == patient]
        connected_doctors = patient_df[patient_df["y"] == 1]["j"].values
        most_popular_doctors = (
            dataframe[dataframe["j"].isin(connected_doctors)]
            .groupby("j")
            .agg({"y": "sum"})
            .sort_values("y", ascending=False)
        )
        not_kept_doctors = most_popular_doctors.index[max_number_connections:].values
        for doctor in not_kept_doctors:
            dataframe.loc[
                (dataframe["i"] == patient) & (dataframe["j"] == doctor), "y"
            ] = 0

        # # We keep the connections choosing randomly
        # patient_df = dataframe[dataframe['i'] == patient]
        # connected_doctors = patient_df[patient_df['y'] == 1]['j'].values
        # not_kept_doctors = np.random.choice(connected_doctors, size=len(connected_doctors) - max_number_connections, replace=False)
        # for doctor in not_kept_doctors:
        #     dataframe[(dataframe['i'] == patient) & (dataframe['j'] == doctor)]['y'] = 0

    # Create a dataframe of fixed effects for patients and doctors, then concatenate it with all the data
    k = len(alpha_graph[0])  # Number of latent factors
    ef_patient = pd.DataFrame(np.zeros((n_patients * n_doctors, k)))
    ef_doctor = pd.DataFrame(np.zeros((n_patients * n_doctors, k)))
    for i in range(k):
        ef_patient_element = []
        ef_doctor_element = []
        ef_patient.rename(columns={i: f"ef_p_{i}"}, inplace=True)
        ef_doctor.rename(columns={i: f"ef_d_{i}"}, inplace=True)
        for j in range(n_patients):

            ef_patient_element += list(np.repeat(alpha_graph[j][i], n_doctors))

        for j in range(n_doctors):
            ef_doctor_element.append(psi_graph[j][i])

        ef_patient[f"ef_p_{i}"] = ef_patient_element
        ef_doctor[f"ef_d_{i}"] = np.tile(ef_doctor_element, n_patients)
    dataframe = pd.concat([dataframe, ef_patient, ef_doctor], axis=1)
    dataframe = dataframe.reset_index().drop(["index"], axis=1)
    return dataframe

In [251]:
df = graph_formation(
    n_patients=100,
    n_doctors=30,
    max_number_connections=5,
    z=0.5,
    type_distance="default",
    beta_age_p_graph=0.01,
    beta_age_d_graph=0.01,
    beta_sex_p_graph=0.1,
    beta_sex_d_graph=0.1,
    beta_distance_graph=-10,
    alpha_law_graph=(-1, 1),
    psi_law_graph=(-1, 1),
    nb_latent_factors=5,
)

In [252]:
df.head()

,i,j,y,age_p,age_d,sex_p,sex_d,distance,ef_p_0,ef_p_1,ef_p_2,ef_p_3,ef_p_4,ef_d_0,ef_d_1,ef_d_2,ef_d_3,ef_d_4
0,0,0,0,29,35,2,1,0.433359,0.74659,-0.753533,-0.659902,-0.81475,-0.573888,-0.820064,0.365443,0.439026,0.208025,-0.485627
1,0,1,0,29,35,2,1,0.779837,0.74659,-0.753533,-0.659902,-0.81475,-0.573888,0.647775,0.118188,0.159054,0.244171,0.651292
2,0,2,0,29,27,2,2,0.531483,0.74659,-0.753533,-0.659902,-0.81475,-0.573888,-0.509647,-0.370625,-0.954527,0.185539,-0.071641
3,0,3,0,29,91,2,2,0.755310,0.74659,-0.753533,-0.659902,-0.81475,-0.573888,-0.370077,-0.762608,-0.745048,0.574609,-0.736623
4,0,4,0,29,95,2,2,0.469799,0.74659,-0.753533,-0.659902,-0.81475,-0.573888,0.691714,0.651430,0.513841,0.476110,0.368256


## MF + Hard Negative Sampling

La fonction graph_formation_hns n'est pas à prendre en compte et me permettait d'effectuer des tests en l'absence d'entraînement initial (estimation initiale des EFs/Betas) pour effectuer le HNS

In [234]:
def graph_formation_hns(
    n_patients,
    n_doctors,
    max_number_connections,
    M=5,
    z=1.4,
    type_distance="default",
    beta_age_p_graph=0.01,
    beta_age_d_graph=0.01,
    beta_sex_p_graph=0.5,
    beta_sex_d_graph=0.5,
    beta_distance_graph=-0.5,
    alpha_law_graph=(-1, 1),
    psi_law_graph=(-1, 1),
    nb_latent_factors=None,
):
    """
    Creates only the graph formation part and returns the associated dataframe
    """
    coor_patients = []
    coor_doctors = []
    alpha_graph = []
    psi_graph = []
    rng = np.random.default_rng(None)
    D = np.zeros([n_patients, n_doctors], dtype=np.ndarray)

    if nb_latent_factors == None:  # We pick a random number of latent factors
        random_number = random.randint(1, 10)
        for i in range(n_patients):

            # We generate the FE for the graph formation model
            alpha_graph.append(
                np.random.uniform(
                    alpha_law_graph[0], alpha_law_graph[1], size=random_number
                )
            )

        for j in range(n_doctors):

            # We generate the FE for the graph formation model
            psi_graph.append(
                np.random.uniform(
                    psi_law_graph[0], psi_law_graph[1], size=random_number
                )
            )

    else:

        for i in range(n_patients):

            # We generate the FE for the graph formation model
            alpha_graph.append(
                np.random.uniform(
                    alpha_law_graph[0], alpha_law_graph[1], size=nb_latent_factors
                )
            )

        for j in range(n_doctors):

            # We generate the FE for the graph formation model
            psi_graph.append(
                np.random.uniform(
                    psi_law_graph[0], psi_law_graph[1], size=nb_latent_factors
                )
            )

    # Generate distance matrix
    if type_distance == "default":
        for i in range(n_patients):
            # Generate the coordinates of the patients
            coor_patients.append(np.random.uniform(0, 1, 2))
            for j in range(n_doctors):
                if (
                    i == 0
                ):  # We ensure each coordinate is generated once for each doctor
                    # Generate the coordinates of the doctors
                    coor_doctors.append(np.random.uniform(0, 1, 2))
                d = np.sqrt(
                    np.power((coor_patients[i][0] - coor_doctors[j][0]), 2)
                    + np.power((coor_patients[i][1] - coor_doctors[j][1]), 2)
                )
                D[i][j] = d

    else:

        # Assign randomly a CODGEO, DEP, or REG to patients and doctors
        dist_matrix = pd.read_csv("../Data/" + type_distance + ".csv")

        del dist_matrix[dist_matrix.columns[0]]
        dist_matrix.index = dist_matrix.columns
        for i in range(len(dist_matrix)):
            dist_matrix.iloc[i, i] = 0
        arr = dist_matrix.columns.values
        for i, col in enumerate(arr):
            arr[i] = int(float(arr[i]))
        dist_matrix.columns = arr
        dist_matrix.index = arr

        # Generate code for patient and doctor
        code_patient = []
        code_doctor = []
        for i in range(n_patients):
            random_code = np.random.choice(dist_matrix.columns.values)
            code_patient.append(random_code)
        for j in range(n_doctors):
            random_code = np.random.choice(dist_matrix.columns.values)
            code_doctor.append(random_code)
        for i in range(n_patients):
            for j in range(n_doctors):
                D[i, j] = dist_matrix.loc[code_patient[i], code_doctor[j]]

    D_normed = (D - D.mean()) / D.std()
    # Random draws of ages for patients and doctors
    sim_patient_age = rng.integers(low=1, high=99, size=n_patients)
    sim_patient_age_normed = (
        sim_patient_age - sim_patient_age.mean()
    ) / sim_patient_age.std()
    sim_doctor_age = rng.integers(low=26, high=99, size=n_doctors)
    sim_doctor_age_normed = (
        sim_doctor_age - sim_doctor_age.mean()
    ) / sim_doctor_age.std()

    # Random draws of genders of patients and doctors
    sim_patient_gender = np.random.choice(np.array([-1, 1]), n_patients)
    sim_doctor_gender = np.random.choice(np.array([-1, 1]), n_doctors)
    # sim_patient_gender = rng.integers(low = 1, high = 3, size = n_patients)
    # sim_doctor_gender = rng.integers(low = 1, high = 3, size = n_doctors)
    sim_patient_gender_normed = (
        sim_patient_gender - sim_patient_gender.mean()
    ) / sim_patient_gender.std()
    sim_doctor_gender_normed = (
        sim_doctor_gender - sim_doctor_gender.mean()
    ) / sim_doctor_gender.std()

    # Bias for patients and doctors
    # bias_patient = np.random.normal(0, 1, size = n_patients)
    # bias_doctor = np.random.normal(0, 1, size = n_doctors)

    # Compile ids
    id_p = np.repeat(range(n_patients), n_doctors)
    id_d = np.tile(range(n_doctors), n_patients)

    # Compile observed features
    age_p_data = np.repeat(sim_patient_age, n_doctors)
    age_d_data = np.tile(sim_doctor_age, n_patients)
    sex_p_data = np.repeat(sim_patient_gender, n_doctors)
    sex_d_data = np.tile(sim_doctor_gender, n_patients)
    # bias_patient_data = np.repeat(bias_patient, n_doctors)
    # bias_doctor_data = np.tile(bias_doctor, n_patients)
    ef_patient_data = np.repeat(alpha_graph, n_doctors)
    ef_doctor_data = np.tile(psi_graph, n_patients)
    if type_distance != "default":
        code_patient_data = np.repeat(code_patient, n_doctors)
        code_doctor_data = np.tile(code_doctor, n_patients)
    # # P is the matrix with all the connection probabilities
    # P = np.zeros((n_patients, n_doctors))
    # Generate the identifier matrix A based on the distance
    A = np.zeros([n_patients, n_doctors], dtype=np.ndarray)
    for i in range(n_patients):
        for j in range(n_doctors):
            if (
                D[i][j] <= z
            ):  # if patient i and doctor j are too far away, there is no relation
                T = (
                    np.dot(alpha_graph[i], psi_graph[j])
                    + beta_age_p_graph * sim_patient_age_normed[i]
                    + beta_age_d_graph * sim_doctor_age_normed[j]
                    + beta_sex_p_graph * sim_patient_gender_normed[i]
                    + beta_sex_d_graph * sim_doctor_gender_normed[j]
                    + beta_distance_graph * D[i][j]
                )
                p = 1 / (1 + np.exp(-T))
                # P[i][j] = p
                A[i][j] = np.random.binomial(1, p)

    # Compile relations between doctors and patients
    relation = A.flatten()

    # Merge all columns into a dataframe
    dataframe = pd.DataFrame(
        data={
            "i": id_p,
            "j": id_d,
            "y": relation,
            "age_p": age_p_data,
            "age_d": age_d_data,
            "sex_p": sex_p_data,
            "sex_d": sex_d_data,
        }
    )
    dataframe["distance"] = D[dataframe["i"], dataframe["j"]].astype(float)

    # Now, we bound the number of connections (1 <= connections <= max_number_connections)
    # First, we detect the patients who have 0 connection.
    number_of_connections = dataframe.groupby("i").agg({"y": "sum"})
    zero_connection = number_of_connections[number_of_connections["y"] == 0].index
    for patient in zero_connection:
        # If patient has zero connection, we connect him with the nearest doctor (even if the threshold z isn't respected)
        min_index = dataframe[dataframe["i"] == patient]["distance"].idxmin()
        doctor_to_connect = dataframe.loc[min_index, "j"]
        dataframe.loc[
            (dataframe["i"] == patient) & (dataframe["j"] == doctor_to_connect), "y"
        ] = 1

    # Then, we detect the patients who have more than max_number_connections. We choose the remaining connections by doctor's popularities (possible to choose randomly).
    number_of_connections = dataframe.groupby("i").agg({"y": "sum"})
    too_much_connection = number_of_connections[
        number_of_connections["y"] > max_number_connections
    ].index
    for patient in too_much_connection:
        # We keep the connections with the most popular doctors
        patient_df = dataframe[dataframe["i"] == patient]
        connected_doctors = patient_df[patient_df["y"] == 1]["j"].values
        most_popular_doctors = (
            dataframe[dataframe["j"].isin(connected_doctors)]
            .groupby("j")
            .agg({"y": "sum"})
            .sort_values("y", ascending=False)
        )
        not_kept_doctors = most_popular_doctors.index[max_number_connections:].values
        for doctor in not_kept_doctors:
            dataframe.loc[
                (dataframe["i"] == patient) & (dataframe["j"] == doctor), "y"
            ] = 0

        # # We keep the connections choosing randomly
        # patient_df = dataframe[dataframe['i'] == patient]
        # connected_doctors = patient_df[patient_df['y'] == 1]['j'].values
        # not_kept_doctors = np.random.choice(connected_doctors, size=len(connected_doctors) - max_number_connections, replace=False)
        # for doctor in not_kept_doctors:
        #     dataframe[(dataframe['i'] == patient) & (dataframe['j'] == doctor)]['y'] = 0

    # Create a dataframe of fixed effects for patients and doctors, then concatenate it with all the data
    k = len(alpha_graph[0])  # Number of latent factors
    ef_patient = pd.DataFrame(np.zeros((n_patients * n_doctors, k)))
    ef_doctor = pd.DataFrame(np.zeros((n_patients * n_doctors, k)))
    for i in range(k):
        ef_patient_element = []
        ef_doctor_element = []
        ef_patient.rename(columns={i: f"ef_p_{i}"}, inplace=True)
        ef_doctor.rename(columns={i: f"ef_d_{i}"}, inplace=True)
        for j in range(n_patients):

            ef_patient_element += list(np.repeat(alpha_graph[j][i], n_doctors))

        for j in range(n_doctors):
            ef_doctor_element.append(psi_graph[j][i])

        ef_patient[f"ef_p_{i}"] = ef_patient_element
        ef_doctor[f"ef_d_{i}"] = np.tile(ef_doctor_element, n_patients)
    dataframe = pd.concat([dataframe, ef_patient, ef_doctor], axis=1)

    # Hard Negative Sampling : à partir de notre dataframe, on doit pouvoir générer des Y = 0 pertinents. Pour cela, il faut effectuer un entrainement initial sur notre dataframe pour
    # avoir une première estimation des paramètres de notre modèle (EF, Betas) et effectuer ce HNS. Cette partie de code n'est pas censée être dans la génération des données,
    # il faut considérer que le HNS s'effectue uniquement à partir de notre dataframe généré, on n'a pas accès aux valeurs théoriques des paramètres (EF, Betas).
    # On enregistre l'ensemble des Y = 0 avant de les supprimer de notre dataframe
    negative_connections = dataframe[dataframe["y"] == 0]
    # We remove all the Y = 0, as we're now using HNS
    dataframe.drop(dataframe[dataframe["y"] == 0].index, inplace=True)
    dataframe = dataframe.reset_index().drop(["index"], axis=1)
    # First, we use the initial training to estimate the scores (probabilities of connection between patients/doctors)
    # parameters = initial_training()
    # alpha_graph_training = parameters[0]
    # psi_graph_training = parameters[1]
    # beta_age_p_graph_training = parameters[2]
    # beta_age_d_graph_training = parameters[3]
    # beta_sex_p_graph_training = parameters[4]
    # beta_sex_d_graph_training = parameters[5]
    # beta_distance_graph_training =  parameters[6]
    alpha_graph_training = alpha_graph
    psi_graph_training = psi_graph
    beta_age_p_graph_training = beta_age_p_graph
    beta_age_d_graph_training = beta_age_d_graph
    beta_sex_p_graph_training = beta_sex_p_graph
    beta_sex_d_graph_training = beta_sex_d_graph
    beta_distance_graph_training = beta_distance_graph
    prediction_scores = np.zeros((n_patients, n_doctors))
    negative_drawing_list = []
    # Estimated scores are based on our MF model
    # On récupère l'ensemble des connexions non existantes (Y = 0)
    negative_n_patients = len(negative_connections["i"].unique())
    highest_prediction_scores_indexes = np.zeros((negative_n_patients, M))
    for i in range(negative_n_patients):
        negative_df = negative_connections[negative_connections["i"] == i]
        # On considère les docteurs pour lesquels le patient i n'a pas de connexions avec
        for j in negative_df["j"]:
            T = (
                np.dot(alpha_graph_training[i], psi_graph_training[j])
                + beta_age_p_graph_training * sim_patient_age_normed[i]
                + beta_age_d_graph_training * sim_doctor_age_normed[j]
                + beta_sex_p_graph_training * sim_patient_gender_normed[i]
                + beta_sex_d_graph_training * sim_doctor_gender_normed[j]
                + beta_distance_graph_training * D[i][j]
            )
            prediction_scores[i][j] = 1 / (1 + np.exp(-T))
        # on garde les M indices des docteurs avec la plus grande probabilité de connexion pour le patient i dans highest_prediction_scores_indexes
        highest_prediction_scores_indexes[i] = np.argsort(prediction_scores[i])[-M:]
        # on effectue le negative sampling
        negative_drawing_patient = []
        # on détermine les y = 0 à rajouter à notre dataframe
        negative_drawing = np.random.binomial(1, p=1 / M, size=M)
        for l in range(M):
            if negative_drawing[l] == 1:
                doc = highest_prediction_scores_indexes[i][l]
                negative_drawing_patient.append(
                    negative_df[negative_df["j"] == doc].iloc[0].to_list()
                )
        negative_drawing_list += negative_drawing_patient
    # Une fois les connexions négatives récupérées pour chaque patient, on concatène nos Y = 1 et Y = 0
    df2 = pd.DataFrame(negative_drawing_list)
    df2.columns = dataframe.columns
    dataframe = pd.concat([dataframe, df2], axis=0)
    dataframe = dataframe.reset_index().drop(["index"], axis=1)

    return dataframe

In [235]:
def hard_negative_sampling(dataframe, M):
    # Hard Negative Sampling : à partir de notre dataframe, on doit pouvoir générer des Y = 0 pertinents. Pour cela, il faut effectuer un entrainement initial sur notre dataframe pour
    # avoir une première estimation des paramètres de notre modèle (EF, Betas) et ensuite effectuer ce HNS.

    # On enregistre l'ensemble des Y = 0 avant de les supprimer de notre dataframe
    negative_connections = dataframe[dataframe["y"] == 0]
    # On supprime les Y = 0 car on va les générer avec le HNS
    dataframe.drop(dataframe[dataframe["y"] == 0].index, inplace=True)
    dataframe = dataframe.reset_index().drop(["index"], axis=1)
    # First, we use the initial training to estimate the scores (probabilities of connection between patients/doctors)
    parameters = initial_training()
    alpha_graph_training = parameters[0]
    psi_graph_training = parameters[1]
    beta_age_p_graph_training = parameters[2]
    beta_age_d_graph_training = parameters[3]
    beta_sex_p_graph_training = parameters[4]
    beta_sex_d_graph_training = parameters[5]
    beta_distance_graph_training = parameters[6]
    prediction_scores = np.zeros((n_patients, n_doctors))
    negative_drawing_list = []
    # Estimated scores are based on our MF model
    negative_patients = negative_connections["i"].unique()
    highest_prediction_scores_indexes = np.zeros((negative_n_patients, M))
    for i in negative_patients:
        negative_df = negative_connections[negative_connections["i"] == i]
        # On considère les docteurs pour lesquels le patient i n'a pas de connexions avec
        for j in negative_df["j"]:
            # on récupère la distance entre le patient i et le docteur j
            patient_df = dataframe[dataframe["i"] == i]
            distance = patient_df[patient_df["j"] == j]["distance"].iloc[0]
            # # We compute predicted scores only if threshold distance is respected, as we consideer there is no interaction if D[i][j] is too high -> NOT FOR HNS, we don't have information on z ?
            # if distance <= z:
            T = (
                np.dot(alpha_graph_training[i], psi_graph_training[j])
                + beta_age_p_graph_training * sim_patient_age_normed[i]
                + beta_age_d_graph_training * sim_doctor_age_normed[j]
                + beta_sex_p_graph_training * sim_patient_gender_normed[i]
                + beta_sex_d_graph_training * sim_doctor_gender_normed[j]
                + beta_distance_graph_training * distance
            )
            prediction_scores[i][j] = 1 / (1 + np.exp(-T))
        # on garde les M indices des docteurs avec la plus grande probabilité de connexion pour le patient i dans highest_prediction_scores_indexes
        highest_prediction_scores_indexes[i] = np.argsort(prediction_scores[i])[-M:]
        # on effectue le negative sampling
        negative_drawing_patient = []
        # on détermine les y = 0 à rajouter à notre dataframe
        negative_drawing = np.random.binomial(1, p=1 / M, size=M)
        for l in range(M):
            if negative_drawing[l] == 1:
                doc = highest_prediction_scores_indexes[i][l]
                negative_drawing_patient.append(
                    negative_df[negative_df["j"] == doc].iloc[0].to_list()
                )
        negative_drawing_list += negative_drawing_patient
    # Une fois les connexions négatives récupérées pour chaque patient, on concatène nos Y = 1 et Y = 0
    df2 = pd.DataFrame(negative_drawing_list)
    df2.columns = dataframe.columns
    dataframe = pd.concat([dataframe, df2], axis=0)
    dataframe = dataframe.reset_index().drop(["index"], axis=1)

    return dataframe

## Descente de gradient alternée

In [236]:
def r_ui(df):
    # à partir du dataset y = 0 et y = 1, obtenir la matrice de connexion
    # df2 = df.copy()
    patients = df["i"].unique()
    n_patients = len(patients)
    n_doctors = len(df2["j"].unique())
    R = np.zeros((n_patients, n_doctors))
    for i in patients:
        for j in df[df["i"] == i]["j"]:
            connection = df[df["i"] == i][df[df["i"] == i]["j"] == j]["y"]
            # if len(connection) > 0:
            if connection[connection.index[0]] == 1:
                R[i][j] = 1
    return R


def alpha(R):
    return (
        np.unique(R.flatten(), return_counts=True)[1][0]
        / np.unique(R.flatten(), return_counts=True)[1][1]
    )  # Nombre de 0 sur nombre de 1


def descente_gradient(
    df, max_iter, eps=0.01, lr_patients=0.001, lr_doctors=0.001, reg_term=0.2
):
    # Je normalise d'abord les colonnes du dataframe
    to_be_normalized_columns = ["age_p", "age_d", "sex_p", "sex_d", "distance"]
    df[to_be_normalized_columns] = (
        df[to_be_normalized_columns] - df[to_be_normalized_columns].mean()
    ) / df[to_be_normalized_columns].std()
    # faire attention, je fais une descente de gradient pas une montée comme dans le papier de recherche, je dois donc calculer x_t = x_(t-1) - ... et pas + ...
    # argmax .. = argmin -... donc je dois prendre le gradient négatif du papier de recherche
    patients = df["i"].unique()
    doctors = df["j"].unique()
    n_patients = len(patients)
    n_doctors = len(doctors)
    nb_latent_factors = len([s for s in df.columns if "ef_p" in s])
    R = r_ui(df)
    alpha_term = alpha(R)
    # We initialise randomly the fixed effects, betas and the bias
    X = np.random.uniform(-1, 1, size=(n_patients, nb_latent_factors))
    Y = np.random.uniform(-1, 1, size=(n_doctors, nb_latent_factors))
    beta_age_doctor = np.random.uniform(-0.5, 0.5)
    beta_age_patient = np.random.uniform(-0.5, 0.5)
    beta_sex_doctor = np.random.uniform(-0.5, 0.5)
    beta_sex_patient = np.random.uniform(-0.5, 0.5)
    beta_distance = np.random.uniform(-0.5, 0.5)
    grad_X = np.zeros(
        (n_patients, nb_latent_factors)
    )  # Matrix containin fixed effects gradients of patients
    grad_Y = np.zeros(
        (n_doctors, nb_latent_factors)
    )  # Matrix containing fixed effects gradients of doctors
    grad_beta_age_doctor = 0
    grad_beta_sex_doctor = 0
    grad_beta_age_patient = 0
    grad_beta_sex_patient = 0
    grad_beta_distance = 0
    sum_squared_sgd_ef_patients = [0 for _ in range(n_patients)]
    sum_squared_sgd_ef_doctors = [0 for _ in range(n_doctors)]
    sum_squared_sgd_beta_age_doctor = 0
    sum_squared_sgd_beta_sex_doctor = 0
    sum_squared_sgd_beta_age_patient = 0
    sum_squared_sgd_beta_sex_patient = 0
    sum_squared_sgd_beta_distance = 0

    k = 0
    first_loop = True  # Allows the while loop to run the first time
    while (
        np.linalg.norm(grad_X) > eps
        or np.linalg.norm(grad_Y) > eps
        or first_loop == True
    ) and k < max_iter:  # Alternate gradient descent
        first_loop = False
        t_0 = time.time()
        k += 1
        grad_beta_age_doctor = 0
        grad_beta_sex_doctor = 0
        for (
            j
        ) in (
            doctors
        ):  # We fix fixed effects / bias of the patients and update the one of the doctors
            grad_Y[j, :] = 0  # We reinitialize at each loop the gradients
            for i in df[df["j"] == j]["i"]:

                sex_patient = df[df["i"] == i]["sex_p"].iloc[0]
                sex_doctor = df[df["j"] == j]["sex_d"].iloc[0]
                age_patient = df[df["i"] == i]["age_p"].iloc[0]
                age_doctor = df[df["j"] == j]["age_d"].iloc[0]
                distance = df[df["i"] == i][df[df["i"] == i]["j"] == j][
                    "distance"
                ].iloc[0]
                # Following terms are defined to optimize calculation time
                exp_term = (1 + alpha_term * R[i][j]) / (
                    1
                    + np.exp(
                        -(
                            np.dot(X[i, :], Y[j, :])
                            + age_patient * beta_age_patient
                            + sex_patient * beta_sex_patient
                            + age_doctor * beta_age_doctor
                            + sex_doctor * beta_sex_doctor
                            + distance * beta_distance
                        )
                    )
                )
                alpha_r = alpha_term * R[i][j]

                grad_Y[j, :] += (
                    -alpha_r * X[i, :] + X[i, :] * exp_term + reg_term * Y[j, :]
                )

                grad_beta_age_doctor += age_doctor * (-alpha_r + exp_term)

                grad_beta_sex_doctor += sex_doctor * (-alpha_r + exp_term)

            # Sum of terms in the denominator for Adaboost
            sum_squared_sgd_ef_doctors[j] += np.dot(grad_Y[j, :], grad_Y[j, :])
            Y[j, :] -= (
                lr_doctors * grad_Y[j, :] / np.sqrt(sum_squared_sgd_ef_doctors[j])
            )

        sum_squared_sgd_beta_age_doctor += grad_beta_age_doctor**2
        sum_squared_sgd_beta_sex_doctor += grad_beta_sex_doctor**2
        beta_age_doctor -= (
            lr_doctors * grad_beta_age_doctor / np.sqrt(sum_squared_sgd_beta_age_doctor)
        )
        beta_sex_doctor -= (
            lr_doctors * grad_beta_sex_doctor / np.sqrt(sum_squared_sgd_beta_sex_doctor)
        )

        grad_beta_age_patient = 0  # We reinitialize at each loop the gradients
        grad_beta_sex_patient = 0
        for (
            i
        ) in (
            patients
        ):  # We now update fixed effects / bias of the patients using our updated fixed effects / bias of doctors
            grad_X[i, :] = 0  # We reinitialize at each loop the gradients
            for j in df[df["i"] == i]["j"]:

                sex_patient = df[df["i"] == i]["sex_p"].iloc[0]
                sex_doctor = df[df["j"] == j]["sex_d"].iloc[0]
                age_patient = df[df["i"] == i]["age_p"].iloc[0]
                age_doctor = df[df["j"] == j]["age_d"].iloc[0]
                distance = df[df["i"] == i][df[df["i"] == i]["j"] == j][
                    "distance"
                ].iloc[0]
                # Following terms are defined to optimize calculation time
                exp_term = (1 + alpha_term * R[i][j]) / (
                    1
                    + np.exp(
                        -(
                            np.dot(X[i, :], Y[j, :])
                            + age_patient * beta_age_patient
                            + sex_patient * beta_sex_patient
                            + age_doctor * beta_age_doctor
                            + sex_doctor * beta_sex_doctor
                            + distance * beta_distance
                        )
                    )
                )
                alpha_r = alpha_term * R[i][j]

                grad_X[i, :] += (
                    -alpha_r * Y[j, :] + Y[j, :] * exp_term + reg_term * X[i, :]
                )

                grad_beta_age_patient += age_patient * (-alpha_r + exp_term)

                grad_beta_sex_patient += sex_patient * (-alpha_r + exp_term)

            sum_squared_sgd_ef_patients[i] += np.dot(
                grad_X[i, :], grad_X[i, :]
            )  # Denominator term for Adaboost
            X[i, :] -= (
                lr_patients * grad_X[i, :] / np.sqrt(sum_squared_sgd_ef_patients[i])
            )

        sum_squared_sgd_beta_age_patient += grad_beta_age_patient**2
        sum_squared_sgd_beta_sex_patient += grad_beta_sex_patient**2
        beta_age_patient -= (
            lr_patients
            * grad_beta_age_patient
            / np.sqrt(sum_squared_sgd_beta_age_patient)
        )
        beta_sex_patient -= (
            lr_patients
            * grad_beta_age_patient
            / np.sqrt(sum_squared_sgd_beta_sex_patient)
        )

        grad_beta_distance = 0
        for (
            i
        ) in (
            patients
        ):  # We finally update the beta_distance as it is dependent on patients and doctors parameters.
            for j in df[df["i"] == i]["j"]:

                sex_patient = df[df["i"] == i]["sex_p"].iloc[0]
                sex_doctor = df[df["j"] == j]["sex_d"].iloc[0]
                age_patient = df[df["i"] == i]["age_p"].iloc[0]
                age_doctor = df[df["j"] == j]["age_d"].iloc[0]
                distance = df[df["i"] == i][df[df["i"] == i]["j"] == j][
                    "distance"
                ].iloc[0]
                exp_term = (1 + alpha_term * R[i][j]) / (
                    1
                    + np.exp(
                        -(
                            np.dot(X[i, :], Y[j, :])
                            + age_patient * beta_age_patient
                            + sex_patient * beta_sex_patient
                            + age_doctor * beta_age_doctor
                            + sex_doctor * beta_sex_doctor
                            + distance * beta_distance
                        )
                    )
                )
                alpha_r = alpha_term * R[i][j]

                grad_beta_distance += distance * (-alpha_r + exp_term)

        sum_squared_sgd_beta_distance += grad_beta_distance**2
        beta_distance -= (
            ((lr_patients + lr_doctors) / 2)
            * grad_beta_distance
            / np.sqrt(sum_squared_sgd_beta_distance)
        )
        t_1 = time.time()

        print(k, "-th iteration took", t_1 - t_0, "to execute.")
    beta = [
        beta_age_doctor,
        beta_sex_doctor,
        beta_age_patient,
        beta_sex_patient,
        beta_distance,
    ]
    grad_beta = [
        grad_beta_age_doctor,
        grad_beta_sex_doctor,
        grad_beta_age_patient,
        grad_beta_sex_patient,
        grad_beta_distance,
    ]
    return ([X, Y, beta], [grad_X, grad_Y, grad_beta])

In [237]:
df = graph_formation(
    n_patients=100,
    n_doctors=30,
    max_number_connections=4,
    z=0.5,
    type_distance="default",
    beta_age_p_graph=0.01,
    beta_age_d_graph=0.01,
    beta_sex_p_graph=0.1,
    beta_sex_d_graph=0.1,
    beta_distance_graph=-10,
    alpha_law_graph=(-1, 1),
    psi_law_graph=(-1, 1),
    nb_latent_factors=5,
)

In [239]:
dataframe = df.copy()
positive_connections = dataframe[dataframe["y"] == 1]
negative_connections = dataframe[dataframe["y"] == 0]
len(negative_connections["i"].unique())

100

In [240]:
# Nombre de non connexions par patient en moyenne
negative_connections["i"].value_counts().mean()

28.69

In [241]:
# M doit être inférieur ou égal à cette valeur
# C'est le plus petit nombre de docteurs auquel un patient i n'est pas connecté
negative_connections["i"].value_counts().min()

26

In [244]:
# On a bien plus de connexions négatives que de connexions positives
positive_connections["i"].value_counts()

i
50    4
22    4
59    3
60    3
86    2
     ..
32    1
31    1
30    1
29    1
99    1
Name: count, Length: 100, dtype: int64

In [245]:
# Nombre de connexions par patient en moyenne
positive_connections["i"].value_counts().mean()

1.31

In [247]:
df = graph_formation_hns(
    n_patients=1000,
    n_doctors=300,
    max_number_connections=5,
    z=0.5,
    M=30,
    type_distance="default",
    beta_age_p_graph=0.01,
    beta_age_d_graph=0.01,
    beta_sex_p_graph=0.1,
    beta_sex_d_graph=0.1,
    beta_distance_graph=-10,
    alpha_law_graph=(-1, 1),
    psi_law_graph=(-1, 1),
    nb_latent_factors=5,
)

In [248]:
df

,i,j,y,age_p,age_d,sex_p,sex_d,distance,ef_p_0,ef_p_1,ef_p_2,ef_p_3,ef_p_4,ef_d_0,ef_d_1,ef_d_2,ef_d_3,ef_d_4
0,0,30,1,86,89,-1,-1,0.367899,-0.682953,-0.815052,0.010601,-0.576334,0.109618,-0.168551,0.065324,0.815440,0.461285,0.983074
1,0,145,1,86,44,-1,1,0.291169,-0.682953,-0.815052,0.010601,-0.576334,0.109618,0.040532,-0.352408,0.731672,-0.114341,-0.670008
2,0,156,1,86,45,-1,1,0.093386,-0.682953,-0.815052,0.010601,-0.576334,0.109618,-0.094269,-0.821354,-0.235760,0.282684,-0.743585
3,0,262,1,86,47,-1,1,0.386003,-0.682953,-0.815052,0.010601,-0.576334,0.109618,-0.971572,0.410381,-0.689428,-0.252345,0.567239
4,0,289,1,86,88,-1,1,0.343987,-0.682953,-0.815052,0.010601,-0.576334,0.109618,-0.391391,-0.904439,0.291136,-0.168596,0.922401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5898,995,275,0,80,67,1,1,0.209940,-0.551933,0.228154,0.829068,-0.545308,-0.901676,0.082150,-0.882055,0.704478,-0.410692,-0.355338
5899,996,132,0,83,93,-1,1,0.253955,-0.489046,0.817064,0.061119,-0.820247,0.267251,-0.488486,0.236925,-0.282689,0.105945,0.081801
5900,997,173,0,2,28,-1,-1,0.015671,0.428782,0.346513,0.196287,-0.351358,0.671217,-0.040852,0.196027,-0.999387,-0.907583,-0.385915
5901,998,97,0,3,56,1,1,0.100773,-0.043227,-0.840932,0.074328,0.421382,-0.467160,-0.709199,0.663311,0.445632,-0.092643,-0.999014


In [249]:
# Comparaison Y = 0 / Y = 1, pas assez d'échantillonnage négatif ?
df["y"].value_counts()

y
1    4946
0     957
Name: count, dtype: int64

In [ ]:
estimation = descente_gradient(
    df=df, eps=0.1, lr_patients=0.01, lr_doctors=0.01, max_iter=100
)

1 -th iteration took 24.55609917640686 to execute.
2 -th iteration took 26.22185707092285 to execute.
3 -th iteration took 25.664959192276 to execute.
4 -th iteration took 23.428571701049805 to execute.
5 -th iteration took 24.576223611831665 to execute.
6 -th iteration took 24.129536867141724 to execute.
7 -th iteration took 24.11001944541931 to execute.
8 -th iteration took 24.294201135635376 to execute.
9 -th iteration took 24.16663360595703 to execute.
10 -th iteration took 24.205181121826172 to execute.
11 -th iteration took 24.013393878936768 to execute.
12 -th iteration took 24.050031900405884 to execute.
13 -th iteration took 24.973216772079468 to execute.
14 -th iteration took 24.470908880233765 to execute.
15 -th iteration took 24.423006772994995 to execute.
16 -th iteration took 23.795681953430176 to execute.
17 -th iteration took 24.83116054534912 to execute.
18 -th iteration took 25.092320680618286 to execute.
19 -th iteration took 25.979527711868286 to execute.
20 -th ite

In [ ]:
df

,i,j,y,age_p,age_d,sex_p,sex_d,distance,ef_p_0,ef_p_1,ef_p_2,ef_p_3,ef_p_4,ef_d_0,ef_d_1,ef_d_2,ef_d_3,ef_d_4
0,0,0,0,-0.724324,-0.879460,-1.061736,0.760759,-1.049642,0.618919,-0.538236,0.529606,0.857231,-0.782108,-0.357861,0.288589,0.522677,0.726029,0.987843
1,0,1,0,-0.724324,-0.060984,-1.061736,0.760759,0.521515,0.618919,-0.538236,0.529606,0.857231,-0.782108,0.442437,-0.808601,0.734503,-0.089774,0.018602
2,0,2,0,-0.724324,-0.831315,-1.061736,0.760759,0.886320,0.618919,-0.538236,0.529606,0.857231,-0.782108,-0.725601,-0.273628,-0.715939,-0.822059,0.216823
3,0,3,0,-0.724324,-1.264625,-1.061736,-1.314038,0.010668,0.618919,-0.538236,0.529606,0.857231,-0.782108,-0.988695,-0.965906,-0.759962,-0.351825,0.637244
4,0,4,0,-0.724324,1.768550,-1.061736,0.760759,0.266071,0.618919,-0.538236,0.529606,0.857231,-0.782108,-0.567096,-0.905338,-0.006534,-0.919454,-0.786606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,99,25,0,0.845300,-0.349858,0.941540,0.760759,-0.162777,-0.045210,-0.007440,-0.268757,0.872298,-0.358166,0.266329,-0.538555,-0.446645,0.366840,-0.500701
2996,99,26,0,0.845300,0.661200,0.941540,0.760759,-0.611306,-0.045210,-0.007440,-0.268757,0.872298,-0.358166,0.403095,0.617180,-0.704687,-0.531276,-0.371224
2997,99,27,0,0.845300,-0.686878,0.941540,0.760759,-0.874019,-0.045210,-0.007440,-0.268757,0.872298,-0.358166,-0.031896,0.581394,-0.248016,0.671538,0.882230
2998,99,28,0,0.845300,-0.638732,0.941540,0.760759,-0.607776,-0.045210,-0.007440,-0.268757,0.872298,-0.358166,0.310468,-0.740340,0.597480,-0.575170,0.848475


In [ ]:
estimation[1][0]

array([[ 7.86510073e-01, -2.93828795e+00, -4.01088596e+00,
         3.32367652e+00, -2.42663156e+00],
       [-6.86205939e+00,  5.33422064e+00,  2.26828245e+00,
        -4.09973338e+00, -6.24010126e+00],
       [ 2.10215838e+00, -8.88230774e+00, -2.36393694e+00,
        -4.92893894e-01,  3.59808490e+00],
       [-1.41164891e+01,  1.00682957e+01,  5.39026517e+00,
         9.08717800e-03, -1.21585999e+01],
       [-5.45473010e+00, -8.26662145e+00,  2.52056527e+00,
         5.63085716e+00, -3.68921928e+00],
       [ 1.13120530e+01, -8.05152227e+00,  4.87896954e+00,
         9.97776168e+00, -1.49861909e+01],
       [ 7.67342151e+00,  1.23532720e+00, -4.37892911e+00,
        -8.56023614e+00,  1.04389170e+01],
       [ 3.28167147e+00, -7.74976258e+00, -1.16264563e+01,
        -1.93535674e+00, -7.49534011e+00],
       [-6.92277611e+00, -5.44984994e-01, -8.88486620e-01,
        -1.39881267e+00, -8.36237379e+00],
       [ 1.16263014e+01, -1.58366721e+00,  9.85928106e+00,
        -6.39932648e+00

In [ ]:
estimation[1][0]

array([[  10.32849808, -161.14226788,  -12.02647753, -190.76972487,
        -146.50364812],
       [ 138.70637802,  180.81759396,  378.32349309, -161.09525635,
         210.0971144 ],
       [ -56.68682564, -388.93916115,  449.26467541,   90.64007968,
        -444.42735107],
       [-577.32402102,   58.095225  , -268.92439174,   70.37177517,
        -513.77491951],
       [ -76.74255115,  215.59410708,  -96.33993459, -479.55691051,
        -541.90797442],
       [ 259.83896305,  315.27726515,  129.40257058, -368.585601  ,
        -349.46143094],
       [-171.15089448,  272.36512943, -430.71971223, -231.95565096,
        -286.45979328],
       [ 200.78194876,  145.89556861,  303.56751626,  116.54376629,
        -133.11359091],
       [-424.65736025, -304.40632085, -199.23753933, -408.79321353,
        -292.65396692],
       [  16.77207877,  -23.683589  , -398.6415066 , -622.71805019,
        -233.3538486 ],
       [ 132.35018445, -189.52201794,  154.67701848, -580.264341  ,
        -476